<img style="float: left; margin: 30px 15px 15px 15px;" src="https://pngimage.net/wp-content/uploads/2018/06/logo-iteso-png-5.png" width="300" height="500" /> 
    
    
### <font color='navy'> Simulación de procesos financieros. 

**Nombres:** Marco Antonio Sánchez Covarrubias. Andrea Flores Fernandez

**Fecha:** 20 de octubre del 2021.

**Expediente** : if720268. if725449
**Profesor:** Oscar David Jaramillo Zuluaga.
    
**Link Github**: https://github.com/MarcoSC08/Tarea7_MASanchez_AFlores

# Tarea 7: Clase 12

# Tarea 

Aproxime el valor de la siguiente integral usando el método monte carlo crudo y método de reducción de varianza de muestreo estratíficado

$$I=\int_{0}^{1}x^2\text{d}x=\left.\frac{x^3}{3}\right|_{x=0}^{x=1}=\frac{1}{3}\approx 0.33333$$

Pasos
1. Cree una función que realice el método de muestreo estratíficado, recibiendo como único parámetro de entrada la cantidad de estratos y retornando las variables estratíficadas correspondientes.
2. Reporte los resultados de la aproximación de la integral usando montecarlo crudo y muestreo estratíficado, en un Dataframe con la información mostrada en la siguiente imagen:
![imagen.png](attachment:imagen.png)


In [1]:
import numpy as np
from functools import reduce
import time
import matplotlib.pyplot as plt
import scipy.stats as st     # Librería estadística
import pandas as pd

Código de solución estudiante 1

In [2]:
# Código de solución estudiante 1
# Integración montecarlo
def int_montecarlo(f:'Función a integrar',
                   a:'Límite inferior de la integral',
                   b:'Límite superior de la integral',
                   U:'Muestra de números U~[a,b]'): # numeros uniformes distribuidos entre a y b
    return (b-a)/len(U)*np.sum(f(U)) # 4 parametros de entrada


In [3]:
I = 1/3 #valor real para compararlo
# intervalo de integración
a = 0; b = 1 

# Cantidad de términos, en escala logarítmica
N = np.logspace(1,7,7,dtype=int) # intervalo de 1-7, 7 valores
#se pone en escala logaritmica porque queremos ir aumentando 10,100,1000,1000....

df = pd.DataFrame(index=N,columns=['Valor_aproximacion', 'Error_relativo1%'], dtype='float')
df.index.name = "Cantidad_terminos"#tabla donde se mostrarán los resultados

# Números aleatorios dependiente de la cantidad de términos N
ui = list(map(lambda N:np.random.uniform(a,b,N),N)) #numeros uniformes 

# Calculamos la aproximación por montecarlo dependiendo de la cantidad de términos que hayamos creado con ui
I_m = list(map(lambda Y:int_montecarlo(lambda x:x**2,a,b,Y),ui)) #funcion, parametros de entrada ,Y, llamamos la funcion con la lista ui

# Mostramos los resultados en la tabla previamente creada
df.loc[N,"Valor_aproximacion"] = I_m
df.loc[N,"Error_relativo1%"] = np.abs(df.loc[N,"Valor_aproximacion"]-I)*100/I
df

,Valor_aproximacion,Error_relativo1%
Cantidad_terminos,,
10,0.223897,32.830906
100,0.309380,7.185947
1000,0.331072,0.678388
10000,0.336610,0.983021
100000,0.333435,0.030518
1000000,0.333326,0.002214
10000000,0.333402,0.020631


In [4]:
np.random.seed(5555)
f=lambda x:x**2
f_inv=lambda u: (3*u)**I

In [5]:
def Muestreo_estratificado(B): #  Muestreo estratificado 
    U2=np.random.rand(B)#B son los # de estratos
    i=np.arange(0,B)
    v=(U2+i)/B
    return v


In [6]:
n=np.logspace(1,7,7,dtype=int)

B=list(map(lambda x: Muestreo_estratificado(x),n))

muestras= list(map(lambda x : f(x),B))
medias = (list(map(lambda x : np.mean(x),muestras)))

In [7]:
df.loc[n,'Muestreo estratificado']=medias
df.loc[n,'Error_relativo2%']=np.abs(df.loc[n,'Muestreo estratificado']-(I))*100/(I)
df

,Valor_aproximacion,Error_relativo1%,Muestreo estratificado,Error_relativo2%
Cantidad_terminos,,,,
10,0.223897,32.830906,0.352906,5.871720e+00
100,0.309380,7.185947,0.333375,1.244718e-02
1000,0.331072,0.678388,0.333325,2.626897e-03
10000,0.336610,0.983021,0.333333,7.690856e-06
100000,0.333435,0.030518,0.333333,3.217504e-06
1000000,0.333326,0.002214,0.333333,6.442973e-08
10000000,0.333402,0.020631,0.333333,8.859913e-10


Código de solución estudiante 2

In [2]:
def int_montecarlo(f:'Función a integrar',
                   a:'Límite inferior de la integral',
                   b:'Límite superior de la integral',
                   U:'Muestra de números U~[a,b]'):
    return (b-a)/len(U)*np.sum(f(U))

In [3]:
N = np.logspace(1,7,7,dtype=int) #cantidad de términos
f = lambda x: x**2
a,b = 0,1
b = 1
u = list(map(lambda N:np.random.uniform(a,b,N),N))
montecarlo = list(map(lambda x:int_montecarlo(f,a,b,x),u))

In [4]:
def estratificar(N):
    global a,b
    def estratos(N):
        U = np.random.uniform(a,b,N)
        segmento = np.arange(0,N)
        muestra = st.uniform.ppf((U + segmento) / N)
        return muestra
    x = [estratos(i) for i in N]
    r = [int_montecarlo(f,a,b,i) for i in x]
    
    return r
estratificar = estratificar(N)

In [5]:
df = pd.DataFrame(index=N)
df.index.name='Cantidad_terminos'
df['Montecarlo crudo'] = montecarlo
df["Error_relativo1%"] = np.abs(df['Montecarlo crudo'] - (1/3)) * (100 / (1/3))
df['Muestreo estratificado'] = estratificar
df["Error_relativo2%"] = np.abs(df['Muestreo estratificado'] - (1/3)) * (100 / (1/3))
df

,Montecarlo crudo,Error_relativo1%,Muestreo estratificado,Error_relativo2%
Cantidad_terminos,,,,
10,0.328702,1.389330,0.332655,2.034766e-01
100,0.342639,2.791833,0.333434,3.015491e-02
1000,0.329482,1.155405,0.333327,1.837953e-03
10000,0.334522,0.356722,0.333333,3.113552e-05
100000,0.334476,0.342665,0.333333,4.463467e-06
1000000,0.333224,0.032794,0.333333,3.807731e-08
10000000,0.333272,0.018399,0.333333,1.894984e-10
